In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hitters/Hitters.csv


In [2]:
hts = pd.read_csv("/kaggle/input/hitters/Hitters.csv")
hts.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [3]:
hts.dropna(inplace=True)

In [32]:
from warnings import filterwarnings
filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict,ShuffleSplit,GridSearchCV
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,RandomForestRegressor,BaseEnsemble,GradientBoostingRegressor
import xgboost 
from xgboost import XGBRegressor
import time

In [5]:
one_hot = pd.get_dummies(hts[["League","Division","NewLeague"]])
one_hot.head()

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
1,0,1,0,1,0,1
2,1,0,0,1,1,0
3,0,1,1,0,0,1
4,0,1,1,0,0,1
5,1,0,0,1,1,0


In [7]:
new_hts = hts.drop(["League","Division","NewLeague","Salary"],axis=1).astype("float64")

In [9]:
X = pd.concat([new_hts,one_hot[["League_N","Division_W","NewLeague_N"]]],axis=1)
X.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,1,1,1
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,0,1,0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,1,0,1
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0,1,0,1
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0,0,1,0


In [10]:
y = hts.Salary

In [11]:
hts.shape

(263, 20)

In [12]:
X.shape

(263, 19)

In [13]:
y.shape

(263,)

In [14]:
X_train = X.iloc[:210]
X_test = X.iloc[210:]
y_train = y[:210]
y_test = y[210:]
print(X_train)
print(X_test)
print(y_train)
print(y_test)

     AtBat   Hits  HmRun  Runs   RBI  Walks  Years  CAtBat   CHits  CHmRun  \
1    315.0   81.0    7.0  24.0  38.0   39.0   14.0  3449.0   835.0    69.0   
2    479.0  130.0   18.0  66.0  72.0   76.0    3.0  1624.0   457.0    63.0   
3    496.0  141.0   20.0  65.0  78.0   37.0   11.0  5628.0  1575.0   225.0   
4    321.0   87.0   10.0  39.0  42.0   30.0    2.0   396.0   101.0    12.0   
5    594.0  169.0    4.0  74.0  51.0   35.0   11.0  4408.0  1133.0    19.0   
..     ...    ...    ...   ...   ...    ...    ...     ...     ...     ...   
257  348.0   90.0   11.0  50.0  45.0   43.0   10.0  2288.0   614.0    43.0   
258  523.0  135.0    8.0  52.0  44.0   52.0    9.0  3368.0   895.0    39.0   
259  312.0   68.0    2.0  32.0  22.0   24.0    1.0   312.0    68.0     2.0   
260  496.0  119.0    8.0  57.0  33.0   21.0    7.0  3358.0   882.0    36.0   
261  126.0   27.0    3.0   8.0  10.0    5.0    4.0   239.0    49.0     3.0   

     CRuns   CRBI  CWalks  PutOuts  Assists  Errors  League_N  

In [15]:
gradient_boost_model = GradientBoostingRegressor().fit(X_train,y_train)

In [22]:
gradient_boost_model

GradientBoostingRegressor()

In [17]:
y_pred = gradient_boost_model.predict(X_train)

In [18]:
y_pred

array([ 495.84268333,  549.6975285 ,  612.72646581,  134.56149569,
        742.07864728,   91.10793842,   95.9841053 ,   91.2116838 ,
       1050.95602769,  533.0082579 ,  528.6056167 ,  512.62181994,
        681.76230548,  325.27383943,  774.39918444,  147.75670885,
        165.80090496,  111.59476248,  124.18730886,  648.66088947,
        800.66846717,  747.97905727,  580.178438  ,  696.11992019,
        699.3671081 ,  954.45928389,  118.41270441,  560.3997828 ,
        284.34374972,  793.22267156,  102.69573735,   87.16734306,
        146.26384744,  296.94375097,  225.57810863,  202.56397796,
        835.96532853,  893.38639112,  101.0339633 , 1147.26311157,
        629.75940363,  311.71357473,  314.9001208 ,  397.14089158,
       1288.68082715,  134.12638738,  281.36386084,  189.75134934,
        217.48742652,  923.5417631 ,   88.5855747 ,  143.8401245 ,
        375.43973892,  812.87500908,  579.11113184,  959.13151767,
        842.00159336,  212.58667359,  506.34474323,  182.49752

In [19]:
np.sqrt(mean_squared_error(y_train,y_pred))

51.9007572535438

In [20]:
r2_score(y_train,y_pred)

0.9875558971028894

In [23]:
y_pred = gradient_boost_model.predict(X_test)

In [24]:
y_pred

array([ 289.62097082,  799.73820237,  188.14333507,  149.86914169,
        136.51234574,  897.08276595,  120.25056393,  408.40655753,
       1207.05401321,  496.14132678,   91.77934091,  434.43635557,
        171.16160236,  241.66338008,  504.53615289,  942.65514389,
        249.4368157 ,  111.05174069,  753.08768892,  937.49328435,
        794.65046856,  579.44156642,  907.83847621,  433.09075012,
        608.33493944,  139.31143144,  964.66473655,  330.41155298,
        720.59266021,  200.88611607, 2024.43538925,  182.92105539,
        279.005558  ,  378.90238166,  823.09548282,  594.95537786,
        509.12415845,  456.36064471,  553.5884331 ,  374.54254707,
        823.89241142,  518.09614153, 1072.66316312,  511.17834835,
       1037.28347513, 1134.6442282 ,  144.45691973,  118.3637193 ,
        669.74010257, 1057.32825294,  587.46934372, 1095.81733532,
        759.52563293])

In [26]:
gradient_boost_params = {"learning_rate":[0.005,0.01,0.1,0.2,0.3],
                            "max_depth":[2,7,15,40,90,110],
                            "n_estimators":[250,600,1000,1900],
                            "subsample":[1,0.5,0.7]}

In [27]:
gbm = GradientBoostingRegressor()

In [29]:
gbm_cv_model = GridSearchCV(gbm,gradient_boost_params,cv=7,n_jobs=-1,verbose=2)

In [36]:
start_time = time.time()

gbm_cv_model.fit(X_train,y_train)

elapsed_time = time.time() - start_time

print(f"Elapsed time for GBM cross validation: "
      f"{elapsed_time:.3f} seconds")

Fitting 7 folds for each of 360 candidates, totalling 2520 fits
[CV] END learning_rate=0.005, max_depth=2, n_estimators=250, subsample=1; total time=   0.3s
[CV] END learning_rate=0.005, max_depth=2, n_estimators=250, subsample=1; total time=   0.3s
[CV] END learning_rate=0.005, max_depth=2, n_estimators=250, subsample=0.5; total time=   0.2s
[CV] END learning_rate=0.005, max_depth=2, n_estimators=250, subsample=0.7; total time=   0.3s
[CV] END learning_rate=0.005, max_depth=2, n_estimators=250, subsample=0.7; total time=   0.3s
[CV] END learning_rate=0.005, max_depth=2, n_estimators=600, subsample=1; total time=   0.7s
[CV] END learning_rate=0.005, max_depth=2, n_estimators=600, subsample=1; total time=   0.7s
[CV] END learning_rate=0.005, max_depth=2, n_estimators=600, subsample=0.5; total time=   0.6s
[CV] END learning_rate=0.005, max_depth=2, n_estimators=600, subsample=0.7; total time=   0.6s
[CV] END learning_rate=0.005, max_depth=2, n_estimators=600, subsample=0.7; total time=  

KeyboardInterrupt: 

In [ ]:
gbm_cv_model.best_params_